# Libraries

In [10]:
import pandas as pd
import pickle
from scipy.stats import poisson
from bs4 import BeautifulSoup
import requests
from string import ascii_uppercase as alphabet

In [11]:
df_fixture = pd.read_csv('/content/df_future.csv')
df_historical_data = pd.read_csv('/content/df_historical_data_kaggle.csv')

# Get Groups

In [12]:
# extracting all tables in website
all_tables = pd.read_html('https://en.wikipedia.org/wiki/2023_FIFA_Women%27s_World_Cup')

In [13]:
all_tables[9]

,Pos,Teamvte,Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,New Zealand (H),0,0,0,0,0,0,0,0,Advance to knockout stage
1,2,Norway,0,0,0,0,0,0,0,0,Advance to knockout stage
2,3,Philippines,0,0,0,0,0,0,0,0,NaN
3,4,Switzerland,0,0,0,0,0,0,0,0,NaN


In [14]:
# A -> H
# starts table 9
# 7 grops
# 7*8 + 9 = 65
for i in range(9,65,7):
    df = all_tables[i]
    df.rename(columns={df.columns[1]:'Team'}, inplace=True)
    df.pop('Qualification')

In [15]:
all_tables = pd.read_html('https://en.wikipedia.org/wiki/2023_FIFA_Women%27s_World_Cup')

dict_table = {}
for letter, i in zip(alphabet, range(9,65,7)):
    df = all_tables[i]
    df.rename(columns={df.columns[1]:'Team'}, inplace=True)
    df.pop('Qualification')
    dict_table[f'Group {letter}'] = df

In [16]:
dict_table.keys()

dict_keys(['Group A', 'Group B', 'Group C', 'Group D', 'Group E', 'Group F', 'Group G', 'Group H'])

In [17]:
dict_table['Group H']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Germany,0,0,0,0,0,0,0,0
1,2,Morocco,0,0,0,0,0,0,0,0
2,3,Colombia,0,0,0,0,0,0,0,0
3,4,South Korea,0,0,0,0,0,0,0,0


# Get Games

In [18]:
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

# Gols Average

In [19]:
df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]

df_home = df_home.rename(columns={'HomeTeam':'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Albania,0.945946,2.891892
Algeria,1.552632,1.657895
American Samoa,0.000000,7.000000
Andorra,0.500000,5.500000
Angola,1.333333,1.733333
...,...,...
West Germany,1.000000,2.500000
Western Australia,0.666667,1.666667
Yugoslavia,0.000000,3.000000


# Methods

## Poisson

In [20]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p

        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

## Get Winner



In [21]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

## Update Result

In [22]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winner {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

# Results

## Groups Fase

In [23]:
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [32]:
dict_table['Group A']

,Team,Pts
0,New Zealand (H),0
1,Norway,0
2,Philippines,0
3,Switzerland,0


## Oitavas


In [24]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winner {group}':group_winner,
                                 f'Runner-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
48,New Zealand (H),Match 49,Costa Rica,2023,?
49,Spain,Match 50,Norway,2023,?
50,United States,Match 51,South Africa,2023,?
51,Sweden,Match 52,Vietnam,2023,?
52,England,Match 54,Republic of Ireland,2023,?
53,Australia (H),Match 53,Haiti,2023,?
54,Germany,Match 56,Jamaica,2023,?
55,France,Match 55,Morocco,2023,?


In [25]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,New Zealand (H),Match 49,Costa Rica,2023,Costa Rica
49,Spain,Match 50,Norway,2023,Norway
50,United States,Match 51,South Africa,2023,United States
51,Sweden,Match 52,Vietnam,2023,Sweden
52,England,Match 54,Republic of Ireland,2023,England
53,Australia (H),Match 53,Haiti,2023,Haiti
54,Germany,Match 56,Jamaica,2023,Germany
55,France,Match 55,Morocco,2023,France


In [26]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,Costa Rica,Match 57,United States,2023,?
57,Norway,Match 58,Sweden,2023,?
58,Haiti,Match 59,France,2023,?
59,England,Match 60,Germany,2023,?


## Quartas

In [27]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Costa Rica,Match 57,United States,2023,United States
57,Norway,Match 58,Sweden,2023,Norway
58,Haiti,Match 59,France,2023,France
59,England,Match 60,Germany,2023,Germany


In [28]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,United States,Match 61,Norway,2023,?
61,France,Match 62,Germany,2023,?


## Semi

In [29]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,United States,Match 61,Norway,2023,United States
61,France,Match 62,Germany,2023,Germany


In [30]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Loser Match 61,Match 63,Loser Match 62,2023,?
63,United States,Match 64,Germany,2023,?


## Final

In [31]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Loser Match 61,Match 63,Loser Match 62,2023,Loser Match 62
63,United States,Match 64,Germany,2023,United States
